In [11]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [17]:
import random
table = ListTable()
table.append(['x', 'y', 'x-y', '(x-y)^2'])
x = random.uniform(0, 10)
y = random.uniform(0, 10)
table.append([x, y, x-y, (x-y)**2])
table

x,y,x-y,(x-y)^2
7.5843365180815905,1.614006730551203,5.970329787530387,35.64483777187264


In [122]:
from IPython.core import magic_arguments
from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class
from IPython.display import HTML, display
import boto3 
import pandas as pd
 
@magics_class
class TestMagics(Magics):
    @cell_magic
    def hello(self, line='', cell=None):
        print('hello ' + cell)
 
    @line_magic
    def hi(self, line):
        print('hi ' + line)
        
    @line_magic
    def db(self, line):
        ec2client = boto3.client('ec2')
        response = ec2client.describe_instances()
        reservations = response.get('Reservations')
        instances = []
        for reservation in reservations:
            reservationInstances = reservation.get('Instances')
            for inst in reservationInstances:
                instances.append(inst)
  
        instancesFormatted = []

        for instance in instances:
            tags = instance.get('Tags', [])
            name = ''
            for tag in tags:
                tagKey = tag.get('Key', '')
                if tagKey == 'Name':
                    name = tag['Value']

            placement = instance['Placement']
            availabilityZone = placement['AvailabilityZone']

            state = instance['State']
            stateName = state.get('Name', '')

            launchTime = instance.get('LaunchTime', '')

            if len(name) > 20:
                name = name[:20] + '...'

            formatInst = {
                'Name': name,
                'Instance Id': instance.get('InstanceId', ''),
                'Instance Type': instance.get('InstanceType', ''),
                'Availability Zone': availabilityZone,
                'State': stateName,
                'Key Name': instance.get('KeyName', ''),
                'Launch Time': launchTime,
            }
            instancesFormatted.append(formatInst)
        display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(key for key in row)) for row in instancesFormatted)
       )
))
        
        display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(row['Availability Zone'])) for row in instancesFormatted)
       )

))
        display(instancesFormatted)
 
ip = get_ipython()
ip.register_magics(TestMagics)

In [123]:
%db

Name,Instance Id,Instance Type,Availability Zone,State,Key Name,Launch Time


u,s,-,w,e,s,t,-,1,c


[{'Availability Zone': 'us-west-1c',
  'Instance Id': 'i-0c7770b48ee09d1ae',
  'Instance Type': 't2.micro',
  'Key Name': 'key_pair_guppi',
  'Launch Time': datetime.datetime(2019, 2, 15, 0, 51, 2, tzinfo=tzutc()),
  'Name': '',
  'State': 'terminated'}]

In [7]:
%hi aaa

hi aaa
